In [1]:
import pandas as pd
import numpy as np

import language_tool_python
_tool_es = language_tool_python.LanguageTool('es')


from pathlib import Path
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_info_columns', 10000)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.2f}'.format)
from typing import List, Dict
from datetime import datetime
import sys
import os
import seaborn as sns
%matplotlib inline
import re

In [2]:
notebook_dir = os.getcwd()
project_root = os.path.join(notebook_dir, '..','..','..','..','..','..','..','..')
sys.path.append(os.path.abspath(project_root))

print("notebook dir:", notebook_dir)
print("project root:", project_root)
print("absolute project root:", os.path.abspath(project_root))
print("notebook dir:", sys.path)

notebook dir: c:\Users\mozac\Documents\proyectos\hitss\claro\bots_rpa\app\modules\sga\minpub\report_validator\service\objetivos\notebooks
project root: c:\Users\mozac\Documents\proyectos\hitss\claro\bots_rpa\app\modules\sga\minpub\report_validator\service\objetivos\notebooks\..\..\..\..\..\..\..\..
absolute project root: c:\Users\mozac\Documents\proyectos\hitss\claro\bots_rpa
notebook dir: ['C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\\python310.zip', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\\DLLs', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\\lib', 'C:\\Users\\mozac\\AppData\\Local\\Microsoft\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0', 'c:\\Users\\mozac\\Documents\\proyectos\\hitss\\claro\\bots_rpa\\venv', '', 'c:\\Users\\mozac\\Documents\\proyectos\\hitss\\claro\\bots_rpa\\venv\\lib\\site-pa

In [3]:
notebook_dir = os.getcwd()
project_root = os.path.join(notebook_dir, '..')
sys.path.append(os.path.abspath(project_root))

sys.path.append(r"")

from utils.logger_config import get_sga_logger
 
logger = get_sga_logger()

In [4]:

def log_exceptions(func):
    """
    Decorator to log exceptions in a function using the shared 'logger'.
    It will also re-raise the exception so that the caller can handle it
    appropriately (e.g., fail fast or continue).
    """
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except Exception as exc:
            logger.error(
                f"Error in function '{func.__name__}': {exc}",
                exc_info=True
            )
            # Optionally, decide whether to re-raise or swallow the exception.
            # Usually best practice is to re-raise so the pipeline can decide what to do:
            raise
    return wrapper



In [5]:
def cut_decimal_part(df, column):
    """
    Converts a DataFrame column from float (or numeric string) to a string
    by removing the decimal part (i.e. converting 13.5 to "13", 12.0 to "12").
    Non-numeric values are converted to NaN and then to an empty string.
    """
    df[column] = pd.to_numeric(df[column], errors='coerce')

    df[column] = df[column].apply(lambda x: str(int(x)) if pd.notnull(x) else '')
    
    return df

In [6]:
def float_to_hhmm(hours_float):
    hours = int(hours_float)
    minutes = int(round((hours_float - hours)*60))
    return f"{hours}:{minutes:02d}"

In [7]:
def seconds_to_hhmm(total_seconds):
    hours = total_seconds // 3600
    minutes = (total_seconds % 3600) // 60
    return f"{hours}:{minutes:02d}"

In [8]:
def handle_null_values(df, fill_str="", fill_float=0.0, fill_datetime=""):
    """
    Fill null values in DataFrame columns based on data type.

    Parameters:
        df (pd.DataFrame): The input DataFrame.
        fill_str (str): Value to replace nulls in object/string columns. Default is "".
        fill_float (float): Value to replace nulls in float columns. Default is 0.0.
        fill_datetime: Value to replace nulls in datetime columns. 
                       Default is "", but you can also pass a default datetime.
    
    Returns:
        pd.DataFrame: The DataFrame with nulls handled.
    """

    obj_cols = df.select_dtypes(include=['object']).columns
    for col in obj_cols:
        df[col] = df[col].fillna(fill_str).astype(str)
    

    float_cols = df.select_dtypes(include=['float64']).columns
    for col in float_cols:
        df[col] = df[col].fillna(fill_float)
        

    datetime_cols = df.select_dtypes(include=['datetime64[ns]']) 
    for col in datetime_cols:
        df[col] = df[col].fillna(fill_datetime)
        
    return df

In [9]:

def resolve_clock_stop_overlaps(clock_stops: List[Dict]) -> List[Dict]:
    """
    Eliminate overlaps in clock stops (paradas de reloj) by nro_incidencia.

    Args:   
        clock_stops: List of clock stops with 'start' 'end' datetime and 'nro_incidencia'

    Returns:
        List of non-overlapping clock stops
            
    """
    if not clock_stops:
        return []
    
    incidents = {}
    for stop in clock_stops:
        nro_incidencia = stop.get('nro_incidencia', 'unknown')
        if nro_incidencia not in incidents:
            incidents[nro_incidencia] = []
        incidents[nro_incidencia].append(stop)

    
    resolved_all = []   

    for nro_incidencia, incident_stops in incidents.items():
        sorted_stops = sorted(incident_stops, key=lambda x: x['start'])

        for i, stop in enumerate(sorted_stops):
            if pd.isna(stop['end']):
                if i < len(sorted_stops) - 1 and not pd.isna(sorted_stops[i+1]['start']):
                    stop['end'] = sorted_stops[i+1]['start']
                else:
                    logger.warning(f"Removing stop with missing end date for nro_incidencia {nro_incidencia}")
                    continue
        
        valid_stops = [stop for stop in sorted_stops if not pd.isna(stop['end'])]

        if not valid_stops:
            continue

        resolved_stops = [valid_stops[0]]

        for current_stop in valid_stops[1:]:
            last_resolved = resolved_stops[-1]

            if current_stop['start'] <= last_resolved['end']:
                last_resolved['end'] = max(last_resolved['end'], current_stop['end'])
            else:
                resolved_stops.append(current_stop)

        resolved_all.extend(resolved_stops)

    return resolved_all

@log_exceptions
def calculate_total_clock_stop_minutes(nro_incidencia:str, interruption_start: datetime, interruption_end: datetime, df_sga_paradas: pd.DataFrame) -> float:
    """
    Calculate the total clock minutes for a ticket, considering constraints.

    Args:
        nro_incidencia: The ticket identifier
        interrupcion_inicio: Start time of the interruption from REPORTE DINAMICO 335 
        interrupcion_fin: End time of the interruption from REPORTE DINAMICO 335 
    
    Returns:
        Total clock stop minutes
    
    """   
    df_sga_paradas['nro_incidencia'] = df_sga_paradas['nro_incidencia'].astype(str)
    nro_incidencia_stops = df_sga_paradas[df_sga_paradas['nro_incidencia'] == nro_incidencia].copy()

    if nro_incidencia_stops.empty:
        logger.info(f"No clock stops found for incident {nro_incidencia}")
        return 0.0
    
    clock_stops = []

    for _, stop in nro_incidencia_stops.iterrows():
        start_date = stop.get('startdate')
        end_date = stop.get('enddate')

        if pd.isna(start_date):
            logger.warning(f"Skipping record with missing start date for incident {nro_incidencia}")
            continue

        if start_date < interruption_start:
            logger.info(f"Adjusting start time to interruption en for incident {nro_incidencia}")
            start_date = interruption_start

        if not pd.isna(end_date):
            if end_date > interruption_end:
                logger.info(f"Adjusting end time to interruption en for incident {nro_incidencia}")
                end_date = interruption_end

            if start_date < end_date:
                clock_stops.append({
                    'start': start_date,
                    'end': end_date,
                    'nro_incidencia': nro_incidencia
                })
        else:
            clock_stops.append({
                'start': start_date,
                'end': end_date,
                'nro_incidencia': nro_incidencia
            })
    resolved_stops = resolve_clock_stop_overlaps(clock_stops)

    total_minutes = sum(
        (stop['end'] - stop['start']).total_seconds() / 60
        for stop in resolved_stops
        if not pd.isna(stop['end']) and not pd.isna(stop['start'])
    )
    return total_minutes



In [10]:
def get_dataframe_summary(df):
    """
    Returns a summary DataFrame for the given DataFrame.
    
    The summary includes:
      - Data Type
      - Non Null Count
      - Null Count
      - Null Percentage
      - Unique Values count
    """
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 1000)
    
    summary_df = pd.DataFrame({
        'Data Type': df.dtypes,
        'Non Null Count': df.count(),
        'Null Count': df.isna().sum(),
        'Null Percentage': (df.isna().sum() / len(df) * 100).round(2),
        'Unique Values': [df[col].nunique() for col in df.columns],
    })
    
    return summary_df

In [11]:
BASE_DIR = Path.cwd().parent.parent.parent.parent.parent.parent.parent.parent
SAVE_DIR_EXTRACT_EXCEL = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "excel"/ "CORTE 2.xlsx"
SAVE_DIR_EXTRACT_SGA_335 = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "sga_335" / "sga_reporte_30-03-2025_06-04-2025_20250410_173936.xlsx"
CID_CUISMP_PATH = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "sharepoint_cid_cuismp" / "MINPU - CID-CUISMP - AB.xlsx"
DIR_PARADAS_RELOJ = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "pausa_cliente" / "sga_reporte_30-03-2025_04-04-2025_20250410_195338.xlsx"

In [12]:
df_corte_excel = pd.read_excel(SAVE_DIR_EXTRACT_EXCEL, skipfooter=2, engine="openpyxl")
df_sga_dinamico_335 = pd.read_excel(SAVE_DIR_EXTRACT_SGA_335) 
df_sga_dinamico_380 = pd.read_excel(DIR_PARADAS_RELOJ)
df_cid_cuismp_sharepoint = pd.read_excel(CID_CUISMP_PATH)

In [13]:
df_sga_dinamico_335['interrupcion_inicio'] = pd.to_datetime(df_sga_dinamico_335['interrupcion_inicio'], errors='coerce', dayfirst=True)
df_sga_dinamico_335['interrupcion_fin'] = pd.to_datetime(df_sga_dinamico_335['interrupcion_fin'], errors='coerce', dayfirst=True)
df_sga_dinamico_335['fecha_comunicacion_cliente'] = pd.to_datetime(df_sga_dinamico_335['fecha_comunicacion_cliente'], errors='coerce', dayfirst=True)
df_sga_dinamico_335['fecha_generacion'] = pd.to_datetime(df_sga_dinamico_335['fecha_generacion'], errors='coerce', dayfirst=True)
df_sga_dinamico_335['fg_padre'] = pd.to_datetime(df_sga_dinamico_335['fg_padre'], errors='coerce', dayfirst=True)
df_sga_dinamico_335['hora_sistema'] = pd.to_datetime(df_sga_dinamico_335['hora_sistema'], errors='coerce', dayfirst=True)
df_sga_dinamico_335["cid"] = df_sga_dinamico_335["cid"].astype(str).fillna("")
df_sga_dinamico_335['nro_incidencia'] = df_sga_dinamico_335['nro_incidencia'].astype(str)
df_sga_dinamico_335 = handle_null_values(df_sga_dinamico_335)
df_sga_dinamico_335["it_determinacion_de_la_causa"] = df_sga_dinamico_335["it_determinacion_de_la_causa"].astype(str).str.strip().fillna('No disponible')
df_sga_dinamico_335["tipo_caso"] = df_sga_dinamico_335["tipo_caso"].astype(str).str.strip().fillna('No disponible')
df_sga_dinamico_335["cid"] = df_sga_dinamico_335["cid"].astype(str).str.strip().fillna('No disponible')
df_sga_dinamico_335 = cut_decimal_part(df_sga_dinamico_335, 'codincidencepadre')

df_sga_dinamico_380['startdate'] = pd.to_datetime(df_sga_dinamico_380['startdate'],  errors='coerce', dayfirst=True)
df_sga_dinamico_380['enddate'] = pd.to_datetime(df_sga_dinamico_380['enddate'],  errors='coerce', dayfirst=True)
df_sga_dinamico_380 = handle_null_values(df_sga_dinamico_380)

df_corte_excel = cut_decimal_part(df_corte_excel,'CUISMP')
#df_corte_excel = cut_decimal_part(df_corte_excel,'CODINCIDENCEPADRE')
df_corte_excel = handle_null_values(df_corte_excel)
df_corte_excel = df_corte_excel.rename(columns={'TICKET':'nro_incidencia'})
df_corte_excel['nro_incidencia'] = df_corte_excel['nro_incidencia'].astype(str)
df_corte_excel['DF'] = df_corte_excel['DF'].astype(str).str.strip().fillna('No disponible').str.lower()
df_corte_excel['CUISMP'] = df_corte_excel['CUISMP'].astype(str).str.strip().fillna('No disponible')
df_corte_excel['DETERMINACIÓN DE LA CAUSA'] = df_corte_excel['DETERMINACIÓN DE LA CAUSA'].astype(str).str.strip().fillna("No disponible")
df_corte_excel['TIPO CASO'] = df_corte_excel['TIPO CASO'].astype(str).str.strip().fillna("No disponible")
df_corte_excel['CID'] = df_corte_excel['CID'].astype(str).str.strip().fillna("No disponible")

df_cid_cuismp_sharepoint = cut_decimal_part(df_cid_cuismp_sharepoint, 'CUISMP')
df_cid_cuismp_sharepoint = df_cid_cuismp_sharepoint.rename(columns={"CID":"cid"})
df_cid_cuismp_sharepoint["cid"] = df_cid_cuismp_sharepoint["cid"].astype(str).fillna("")
df_cid_cuismp_sharepoint["Distrito Fiscal"] = df_cid_cuismp_sharepoint["Distrito Fiscal"].astype(str).str.strip().fillna('No disponible').str.lower()
df_cid_cuismp_sharepoint["CUISMP"] = df_cid_cuismp_sharepoint["CUISMP"].astype(str).str.strip().fillna('No disponible')

In [14]:
def merge_sga_335_corte_excel_sharepoint_cuismp_sga380(
        df_corte_excel: pd.DataFrame, 
        df_sga_dinamico_335: pd.DataFrame,
        df_cid_cuismp_sharepoint: pd.DataFrame,
        df_sga_dinamico_380: pd.DataFrame,
        match_type:str
    ) -> pd.DataFrame:
        """
        Common merge function for Objective 1.

        Merges:
          - corte-excel  with sga_dinamico_335 on 'nro_incidencia'

        Returns a merged DataFrame with common columns needed.
        """

        merged_sga335_excel = pd.merge(
            df_corte_excel,
            df_sga_dinamico_335,
            on='nro_incidencia',
            how='left',
            indicator=True,
            suffixes=('_corte_excel', '_sga_dinamico_335')
        )

        merge_sga_335_corte_excel_matched_with_sharepoint_cid_cuismp = pd.merge(
        merged_sga335_excel,
        df_cid_cuismp_sharepoint,
        on='cid',
        how='left',
        suffixes=('_sga_dinamico_335_excel_matched', '_sharepoint_cid_cuismp')
        )

        merge_sga_335_corte_excel_matched_with_sharepoint_cid_cuismp['sum_paradas'] = merge_sga_335_corte_excel_matched_with_sharepoint_cid_cuismp.apply(
            lambda r: calculate_total_clock_stop_minutes(
                nro_incidencia = r["nro_incidencia"],
                interruption_start = r["interrupcion_inicio"],
                interruption_end = r["interrupcion_fin"],
                df_sga_paradas = df_sga_dinamico_380
            ),
            axis= 1
        )

        matched_rows = merge_sga_335_corte_excel_matched_with_sharepoint_cid_cuismp[merge_sga_335_corte_excel_matched_with_sharepoint_cid_cuismp['_merge'] == match_type]

        return matched_rows

In [15]:
df_matched_corte_sga335_Sharepoint_cuismp_sga380 = merge_sga_335_corte_excel_sharepoint_cuismp_sga380(
        df_corte_excel, df_sga_dinamico_335,
        df_cid_cuismp_sharepoint, df_sga_dinamico_380,
        'both'
        )
df_unmatched_corte_sga335_Sharepoint_cuismp_sga380 = merge_sga_335_corte_excel_sharepoint_cuismp_sga380(
    df_corte_excel,
    df_sga_dinamico_335,
    df_cid_cuismp_sharepoint,
    df_sga_dinamico_380,
    'left_only'
    )


No clock stops found for incident 21789759
No clock stops found for incident 21789943
No clock stops found for incident 21790147
No clock stops found for incident 21790467
No clock stops found for incident 21790487
No clock stops found for incident 21790550
No clock stops found for incident 21790828
No clock stops found for incident 21791018
No clock stops found for incident 21791311
No clock stops found for incident 21791695
No clock stops found for incident 21792007
No clock stops found for incident 21792226
No clock stops found for incident 21792345
No clock stops found for incident 21792390
No clock stops found for incident 21792405
No clock stops found for incident 21792407
No clock stops found for incident 21792446
No clock stops found for incident 21792453
No clock stops found for incident 21792494
No clock stops found for incident 21792500
No clock stops found for incident 21792504
No clock stops found for incident 21792509
No clock stops found for incident 21792537
No clock st

In [16]:
df_matched_corte_sga335_Sharepoint_cuismp_sga380.head(1)

,nro_incidencia,FECHA Y HORA INICIO,FECHA Y HORA FIN,ESTATUS,SERVICIO,CUISMP_sga_dinamico_335_excel_matched,TIPO CASO,AVERÍA,TIEMPO (HH:MM),COMPONENTE,DF,DIRECCION,OBSERVACIÓN,CID,FIN-INICIO (HH:MM),DETERMINACIÓN DE LA CAUSA,RESPONSABILIDAD,TIPO REPORTE,Duracion entero,Agrupación entero,USUARIO,ESPECIALISTA,CODINCIDENCEPADRE,MASIVO,MEDIDAS CORRECTIVAS Y/O PREVENTIVAS TOMADAS,TIPO DE INCIDENCIA,TIEMPO INTERRUPCION,INDISPONIBILIDAD,DC + INDISPONIBILIDAD,canal_ingreso,filter,codincidencepadre,masivo,hijos,pxav,estado,mesa_atencion,catg,ubigeo,cnoci,departamento,distrito,caso,tipo_caso,nombre_cliente,codigo_cliente,sector,segm,cid,mediotxcid,mediotxide,mediotxsot,mediotx,direccion,tipopy,srv,um,dias_transcurridos,considerar,comentario,otpint,sotpint,fecgensotpint,pausacliente2pint,otpext,sotpext,fecgensotpext,sotwimax,sotlte,crq,cantidadcrq,seqcrq,crqobs,remedy,site,cantidadremedy,seqremedy,remedyobs,sot,usot,feccomsot,sot_comentario,compute_0055,fo,mo,slacomp,desp,desplazamiento,horas_pendiente,horas_sin_update,tipo_incidencia,tipo_servicio,producto,bandeja_cnoc,dpto_actual,fecha_comunicacion_cliente,fecha_generacion,fg_padre,fecha_instalacion,fecha_apertura,mes_apertura,anio_apertura,dia_apertura,semana_del_mes,hora,hl,dpto_origen,dpto_origen_detalle,area_1ra_anotacion,area_que_derivo_al_cnoc,derivadoalcnoc,demora_derivacion,demora_tomar_ticket,usuariotomaticket,proactivo,usuario_actual,usuario_ultima_anotacion,compute_0090,compute_0091,derivacion_del_operador,area_ultima_anotacion,fecha_derivacion_cierre,mes_derivacion_cierre,dia_derivacion_cierre,usuario_cierra,tipo_usuario_cierra,tipificacion_interrupcion,tip_interr_filtrado,interrupcion_inicio,interrupcion_fin,tiempocliente,tiempo_interrupcion,compute_0105,compute_0106,compute_0107,compute_0108,compute_0109,compute_0110,compute_0111,compute_0112,tiempopint,tiempored,tiempoproveedor,tiempopext,cruce,tipificacion_problema,tipificacion_observacion,tipificacion_tipo,tipificacion_responsable,tiempotransitocnoc,it_userid,it_responsable,it_determinacion_de_la_causa,it_medidas_tomadas,it_conclusiones,reinc,hora_sistema,_merge,CUISMP_sharepoint_cid_cuismp,Distrito Fiscal,%Disponibilidad,BW contratado,SEDE,CID NUEVO,sum_paradas
1,21789943,2025-03-18 16:46:00,2025-03-19 12:54:00,Caso Concluído,Red Privada Virtual Full Mesh,14008,SIN SERVICIO,BLOQUEO DE MODEM SATELITAL,11:49:00,COMPONENTE II,junin,"SN . . SIN NOMBRE Jr. Huancayo Plaza Principal de Santo Domingo de Acobamba (FISCALIA , DML) / -11.768895,-74.795526",Se generó ticket para la revisión del servicio de datos de la sede Acobamba2,21092931,20:08:00,El inconveniente se originó debido a un problema con el equipo de red satelital ubicado en la sedel cliente.,CLARO,RECLAMO,11,Entre 8h a 24h,E756566,CLARO - CORTE,21789943,No,"El cliente, el Sr. Denis Carhuamaca, reportó problemas con el servicio de datos identificado con el CUISMP 14008 y se generó el ticket 21789943 el día 18/03/2025 a las 16:46 horas. Inmediatamente, Claro realizó las revisiones encontrando un inconveniente de conectividad con los equipos ubicados en la sede del cliente. Ante ello, intentamos comunicarnos con el cliente, el Sr. Denis Carhuamaca a las 17:24 horas, sin éxito. Posteriormente, a las 19:04 horas en comunicación el cliente, el Sr. Denis Carhuamaca, se realizó los descartes correspondientes de reinicio de equipos y validación de conexiones, sin superar el evento de pérdida de conectividad. Seguidamente, se gestionó el desplazamiento de personal técnico especializado a la sede del cliente para las revisiones correspondientes, durante la gestión de desplazamiento se verificó el restablecimiento del servicio el día 19/03/2025 a las 12:54 horas por restablecimiento en el equipo de red satelital ubicado en la sedel cliente. Finalmente, el desplazamiento de personal a la sede del cliente se revisará con el ticket 21790828.",REPORTE PREVIO - Calidad,"70,56",Se tuvo indisponibilidad por parte del cliente para continuar los trabajos el/los día(s) \n18/03/2025 22:

In [17]:
info =  get_dataframe_summary(df_matched_corte_sga335_Sharepoint_cuismp_sga380)
info


,Data Type,Non Null Count,Null Count,Null Percentage,Unique Values
nro_incidencia,object,136,0,0.00,136
FECHA Y HORA INICIO,datetime64[ns],136,0,0.00,132
FECHA Y HORA FIN,datetime64[ns],136,0,0.00,107
ESTATUS,object,136,0,0.00,2
SERVICIO,object,136,0,0.00,4
CUISMP_sga_dinamico_335_excel_matched,object,136,0,0.00,102
TIPO CASO,object,136,0,0.00,5
AVERÍA,object,136,0,0.00,23
TIEMPO (HH:MM),object,136,0,0.00,125
COMPONENTE,object,136,0,0.00,3


In [21]:
@log_exceptions
def validation_medidas_correctivas(merged_df: pd.DataFrame) -> pd.DataFrame:
    """
    Validation the column medidas correctivas y o medidas tomadas, se debe obtener
    the first and the last date from paragraph, excluding the two last lines if exists 
    dates called fecha hora inicio and fecha hora fin
    """

    df = merged_df.copy()

    df['mc_first_ok'] = True
    df['mc_last_ok'] = True
    df['it_first_ok'] = True
    df['it_last_ok'] = True
    df['ortografia_ok'] = True
    df['no_repeticion_ok'] = True


    @log_exceptions
    def extract_date_range_last(text: str):
        """
        Devuelve (fecha_hora_inicio, fecha_hora_fin) extraídas
        de las dos últimas líneas no vacías que empiecen con
        'Fecha y hora inicio:' y 'Fecha y hora fin:'.
        Si falla el parseo, retorna (None, None).
        """
        if not isinstance(text, str):
            return (None, None)

        # limpia retornos de carro literales
        text = text.replace('\r', ' ')

        lines = [ln.strip() for ln in text.splitlines() if ln.strip()]
        if len(lines) < 2:
            return (None, None)
        start_line, end_line = lines[-2], lines[-1]

        # fecha obligatorio, “a las” opcional, hora obligatorio, “horas” opcional
        pattern = (
            r'^(?:Fecha y Hora|Hora)(?:\s+de)?\s+'    # “Fecha y Hora” o “Hora”, con “ de” opcional
            r'(Inicio|Fin):\s*'                      # “Inicio:” o “Fin:”
            r'(\d{1,2}/\d{1,2}/\d{4})'                # grupo(2) = fecha
            r'\s*(?:a las\s*)?'                      # “a las” opcional
            r'(\d{1,2}:\d{2})'                       # grupo(3) = hora
            r'(?:\s+horas\.?)?'                      # “horas” opcional
        )
        rx = re.compile(pattern, re.IGNORECASE)

        def parse(line):
            m = rx.match(line)
            if not m:
                return None
            # m.group(1) es “Inicio” o “Fin”  
            fecha, hora = m.group(2), m.group(3)
            return f"{fecha} {hora}"

        inicio = parse(start_line)
        fin    = parse(end_line)
        if inicio is None or fin is None:
            return (None, None)
        return (inicio, fin)


    @log_exceptions
    def extract_date_range_body(text: str):
        if not isinstance(text, str):
            return (None, None)
        
        lines = [ln.strip()  for ln in text.splitlines() if ln.strip()]

        def is_meta_line(ln):
            low = ln.lower()
            return low.startswith("fecha y hora inicio") or low.startswith("fecha y hora fin")
        
        while len(lines) and is_meta_line(lines[-1]):
            lines.pop()

        body = "\n".join(lines)

        pattern = r"(\d{2}/\d{2}/\d{4})\s*(?:a las\s*)?(\d{2}:\d{2})"
        matches = re.findall(pattern, body, flags=re.IGNORECASE)

        if not matches:
            return (None, None)
        
        first_date, first_time = matches[0]
        last_date, last_time = matches[-1]

        return (f"{first_date} {first_time}", f"{last_date} {last_time}")
    
    @log_exceptions
    def has_repetition(text: str) -> bool:
        """
        Returns True if text contains o forbiden repetition of:
        - "Inmediatamente" twice, or
        - "A través" twice.
        """
        if not isinstance(text, str):
            return False
        
        patterns = [
            r'(?i)\b(inmediatamente)\b.*\b\1\b',
            r'(?i)\b(a través)\b.*\b\1\b',
        ]
        return any(re.search(p, text) for p in patterns )
   
    @log_exceptions
    def is_langtool_clean(text:str) -> bool:
        """
        Return True if LanguageTool reports zero issues in the text.
        """
        if not isinstance(text, str) or not text.strip():
            return True
        
        matches = _tool_es.check(text)
        return len(matches) == 0
    

    date_range_mc_body = df['MEDIDAS CORRECTIVAS Y/O PREVENTIVAS TOMADAS'].apply(extract_date_range_body)
    df[['first_dt_mc', 'last_dt_mc']] = pd.DataFrame(date_range_mc_body.tolist(), index=df.index)

    date_range_it_body = df['it_medidas_tomadas'].apply(extract_date_range_body)
    df[['first_dt_it', 'last_dt_it']] = pd.DataFrame(date_range_it_body.tolist(), index=df.index)

    date_range_it_last = df['it_medidas_tomadas'].apply(extract_date_range_last)
    df[['start_dt_last_it', 'end_dt_last_it']] = pd.DataFrame(date_range_it_last.tolist(), index=df.index)

    
    df['FECHA_Y_HORA_INICIO_fmt'] = (
        df['FECHA Y HORA INICIO']
        .dt.strftime('%d/%m/%Y %H:%M')
        .fillna("N/A")
        .astype(str)
    )

    df['FECHA_Y_HORA_FIN_fmt'] = (
        df['FECHA Y HORA FIN']
        .dt.strftime('%d/%m/%Y %H:%M')
        .fillna("N/A")
        .astype(str)
    )
    
    df['mc_first_ok'] = (
        df['first_dt_mc'] == df['FECHA_Y_HORA_INICIO_fmt']
    )

    df['mc_last_ok'] = (
        df['last_dt_mc'] == df['FECHA_Y_HORA_FIN_fmt']
    )

    df['it_first_ok'] = (
        df['first_dt_it'] == df['start_dt_last_it']
    )

    df['it_last_ok'] = (
        df['last_dt_it'] == df['end_dt_last_it']
    )

    df['ortografia_ok'] = ~df['MEDIDAS CORRECTIVAS Y/O PREVENTIVAS TOMADAS'].apply(is_langtool_clean)
    df['no_repeticion_ok'] = ~df['MEDIDAS CORRECTIVAS Y/O PREVENTIVAS TOMADAS'].apply(has_repetition)

 
    df['Validation_OK'] = (
        df['mc_first_ok'] &
        df['mc_last_ok'] &
        df['it_first_ok'] &
        df['it_last_ok'] &
        df['ortografia_ok'] &
        df['no_repeticion_ok'] 
    )

    df['fail_count'] = ( 
        (~df['mc_first_ok']).astype(int)+
        (~df['mc_last_ok']).astype(int)+
        (~df['it_first_ok']).astype(int)+
        (~df['it_last_ok']).astype(int)+
        (~df['ortografia_ok']).astype(int)+ 
        (~df['no_repeticion_ok']).astype(int)           
    )

    return df

df_validation = validation_medidas_correctivas(df_matched_corte_sga335_Sharepoint_cuismp_sga380)
df_validation.head(1)
prueba = df_validation[df_validation['nro_incidencia'] == '21790461']
prueba

,nro_incidencia,FECHA Y HORA INICIO,FECHA Y HORA FIN,ESTATUS,SERVICIO,CUISMP_sga_dinamico_335_excel_matched,TIPO CASO,AVERÍA,TIEMPO (HH:MM),COMPONENTE,DF,DIRECCION,OBSERVACIÓN,CID,FIN-INICIO (HH:MM),DETERMINACIÓN DE LA CAUSA,RESPONSABILIDAD,TIPO REPORTE,Duracion entero,Agrupación entero,USUARIO,ESPECIALISTA,CODINCIDENCEPADRE,MASIVO,MEDIDAS CORRECTIVAS Y/O PREVENTIVAS TOMADAS,TIPO DE INCIDENCIA,TIEMPO INTERRUPCION,INDISPONIBILIDAD,DC + INDISPONIBILIDAD,canal_ingreso,filter,codincidencepadre,masivo,hijos,pxav,estado,mesa_atencion,catg,ubigeo,cnoci,departamento,distrito,caso,tipo_caso,nombre_cliente,codigo_cliente,sector,segm,cid,mediotxcid,mediotxide,mediotxsot,mediotx,direccion,tipopy,srv,um,dias_transcurridos,considerar,comentario,otpint,sotpint,fecgensotpint,pausacliente2pint,otpext,sotpext,fecgensotpext,sotwimax,sotlte,crq,cantidadcrq,seqcrq,crqobs,remedy,site,cantidadremedy,seqremedy,remedyobs,sot,usot,feccomsot,sot_comentario,compute_0055,fo,mo,slacomp,desp,desplazamiento,horas_pendiente,horas_sin_update,tipo_incidencia,tipo_servicio,producto,bandeja_cnoc,dpto_actual,fecha_comunicacion_cliente,fecha_generacion,fg_padre,fecha_instalacion,fecha_apertura,mes_apertura,anio_apertura,dia_apertura,semana_del_mes,hora,hl,dpto_origen,dpto_origen_detalle,area_1ra_anotacion,area_que_derivo_al_cnoc,derivadoalcnoc,demora_derivacion,demora_tomar_ticket,usuariotomaticket,proactivo,usuario_actual,usuario_ultima_anotacion,compute_0090,compute_0091,derivacion_del_operador,area_ultima_anotacion,fecha_derivacion_cierre,mes_derivacion_cierre,dia_derivacion_cierre,usuario_cierra,tipo_usuario_cierra,tipificacion_interrupcion,tip_interr_filtrado,interrupcion_inicio,interrupcion_fin,tiempocliente,tiempo_interrupcion,compute_0105,compute_0106,compute_0107,compute_0108,compute_0109,compute_0110,compute_0111,compute_0112,tiempopint,tiempored,tiempoproveedor,tiempopext,cruce,tipificacion_problema,tipificacion_observacion,tipificacion_tipo,tipificacion_responsable,tiempotransitocnoc,it_userid,it_responsable,it_determinacion_de_la_causa,it_medidas_tomadas,it_conclusiones,reinc,hora_sistema,_merge,CUISMP_sharepoint_cid_cuismp,Distrito Fiscal,%Disponibilidad,BW contratado,SEDE,CID NUEVO,sum_paradas,mc_first_ok,mc_last_ok,it_first_ok,it_last_ok,ortografia_ok,no_repeticion_ok,first_dt_mc,last_dt_mc,first_dt_it,last_dt_it,start_dt_last_it,end_dt_last_it,FECHA_Y_HORA_INICIO_fmt,FECHA_Y_HORA_FIN_fmt,Validation_OK,fail_count
3,21790461,2025-03-19 15:11:00,2025-03-24 13:00:00,Caso Concluído,Telefonía Fija,19001,SIN SERVICIO-NO DA TONO,ANEXO DESCONFIGURADO,12:42:00,COMPONENTE IV,lima noroeste,"SN . . SIN NOMBRE Mz. C1 Lt. 8 Urb. Ex Zona Industrial / Sede Central/Corporativa / -11.869428,-77.128610",Se generó ticket para la revisión del servicio de telefonía de la sede Ventanilla1,21097366,117:49,El inconveniente se originó por un problema en la configuración de los anexos ubicados en la sede del cliente.,CLARO,RECLAMO,12,Entre 8h a 24h,E753186,CLARO - CORTE GESTIONADO,21790461,No,"El cliente, el Sr. Robert Leon, reportó inconvenientes con los anexos de la sede identificada con el CUISMP 19001 y se generó un ticket el día 19/03/2025 a las 15:11 horas. Inmediatamente, Claro se comunicó con el cliente para coordinar una ventana de trabajo y según la disponibilidad del cliente se programó la revisión para el día 24/03/2025 a las 08:00 horas. Luego de las revisiones correspondientes por parte del personal técnico asignado, se encontró los anexos desconfigurados, por lo cual se procedió con la reconfiguración. Seguidamente en base a la disponibilidad del cliente se realizó la validación de los anexos. Finalmente, luego de los correctivos, se verificó el correcto funcionamiento y estabilidad del servicio el 24/03/2025 a las 13:00 horas.",REPORTE PREVIO - Calidad,"12,70",Se tuvo indisponibilidad por parte del cliente para continuar los trabajos el/los día(s) \n19/03/2025 22:43:00 hasta el día 24/03/2025 8:00:00\n(Total de horas sin acceso a la sede: 105:17 horas),El inco

In [22]:
@log_exceptions
def build_failure_messages_medidas_correctivas(df:pd.DataFrame) -> pd.DataFrame:
    """
    Build detailed error messages for medidas correctivas validation failures.

    """

    if not isinstance(df, pd.DataFrame) or df.empty or 'Validation_OK' not in df.columns:
        return pd.DataFrame(columns=['nro_incidencia', 'mensaje', 'TIPO REPORTE','objetivo'])
    
    mensajes = np.where(
       df['Validation_OK'],
       "Validación exitosa: MEDIDAS CORRECTIVAS Y/O PREVENTIVAS TOMADAS",
       (
                     np.where(~df['mc_first_ok'],
                    " La fecha/hora de inicio del parrafo en MEDIDAS CORRECTIVAS:  ( " + df['first_dt_mc'].astype(str) +
                      " ) no coincide con la columna FECHA Y HORA INICIO DE EXCEL: " + df['FECHA_Y_HORA_INICIO_fmt'].astype(str) + ". ",
                    "") +

                      np.where(~df['mc_last_ok'],
                    " La fecha/hora de fin del parrafo en MEDIDAS CORRECTIVAS:( " + df['last_dt_mc'].astype(str) +
                      " ) no coincide con la columna FECHA Y HORA FIN DE EXCEL: " +
                      df['FECHA_Y_HORA_FIN_fmt'].astype(str)+ ". ", 
                    "") + 

                      np.where(~df['it_first_ok'],
                    " La fecha de inicio del parrafo en it_medidas_tomadas:  ( " + df['first_dt_it'].astype(str) +
                      " ) no coincide con la fecha inicio de la penultima fila: " + df['start_dt_last_it'].astype(str) + ". ",
                    "") +

                     np.where(~df['it_last_ok'],
                    " La fecha de fin del parrafo en it_medidas_tomadas:( " + df['last_dt_mc'].astype(str) +
                      " ) no coincide con la fecha fin de la ultima fila: " +
                      df['end_dt_last_it'].astype(str) + ". ", 
                    "") + 

                    np.where(~df['ortografia_ok'],
                    "  Errores ortográficos/gramaticales en el parrafo en MEDIDAS CORRECTIVAS",
                    "")  +

                    np.where(~df['no_repeticion_ok'],
                    " Hay palabras repetidas inmediatamente/a través en el parrafo en MEDIDAS CORRECTIVAS:",
                    "") 
       )
    )
    
    df['mensaje'] = mensajes
    df['objetivo'] = 1.8
    df_failures = df[df['fail_count'] > 0]
    return df_failures[['nro_incidencia', 'mensaje', 'TIPO REPORTE','objetivo']]

df_failures = build_failure_messages_medidas_correctivas(df_validation)
df_failures
    




,nro_incidencia,mensaje,TIPO REPORTE,objetivo
7,21790524,La fecha/hora de fin del parrafo en MEDIDAS CORRECTIVAS:( 21/03/2025 15:03 ) no coincide con la columna FECHA Y HORA FIN DE EXCEL: 27/03/2025 11:30. La fecha de inicio del parrafo en it_medidas_tomadas: ( 19/03/2025 16:22 ) no coincide con la fecha inicio de la penultima fila: 21/03/2025 14:42.,RECLAMO,1.80
15,21791311,La fecha/hora de fin del parrafo en MEDIDAS CORRECTIVAS:( 28/03/2023 10:12 ) no coincide con la columna FECHA Y HORA FIN DE EXCEL: 21/03/2025 17:40. La fecha de fin del parrafo en it_medidas_tomadas:( 28/03/2023 10:12 ) no coincide con la fecha fin de la ultima fila: 21/03/2025 17:40.,RECLAMO,1.80
35,21795643,La fecha/hora de fin del parrafo en MEDIDAS CORRECTIVAS:( 03/31/2025 08:08 ) no coincide con la columna FECHA Y HORA FIN DE EXCEL: 31/03/2025 08:08.,PROACTIVO,1.80
36,21796219,La fecha de fin del parrafo en it_medidas_tomadas:( 31/03/2025 06:30 ) no coincide con la fecha fin de la ultima fila: 31/03/2025 06:30.,PROACTIVO,1.80
40,21796282,La fecha/hora de fin del parrafo en MEDIDAS CORRECTIVAS:( 15/03/2025 15:30 ) no coincide con la columna FECHA Y HORA FIN DE EXCEL: 31/03/2025 15:30. La fecha de fin del parrafo en it_medidas_tomadas:( 15/03/2025 15:30 ) no coincide con la fecha fin de la ultima fila: 31/03/2025 15:30.,RECLAMO,1.80
41,21796323,La fecha/hora de fin del parrafo en MEDIDAS CORRECTIVAS:( 31/03/2025 16:00 ) no coincide con la columna FECHA Y HORA FIN DE EXCEL: 31/03/2025 17:54. La fecha de inicio del parrafo en it_medidas_tomadas: ( 31/03/2025 10:12 ) no coincide con la fecha inicio de la penultima fila: None. La fecha de fin del parrafo en it_medidas_tomadas:( 31/03/2025 16:00 ) no coincide con la fecha fin de la ultima fila: None.,RECLAMO,1.80
43,21796489,La fecha/hora de fin del parrafo en MEDIDAS CORRECTIVAS:( 01/04/2025 11:09 ) no coincide con la columna FECHA Y HORA FIN DE EXCEL: 01/04/2025 11:06. La fecha de fin del parrafo en it_medidas_tomadas:( 01/04/2025 11:09 ) no coincide con la fecha fin de la ultima fila: 01/04/2025 11:09.,PROACTIVO,1.80
44,21796518,La fecha de inicio del parrafo en it_medidas_tomadas: ( 31/03/2025 14:49 ) no coincide con la fecha inicio de la penultima fila: 32/03/2025 14:49. La fecha de fin del parrafo en it_medidas_tomadas:( 31/03/2025 16:43 ) no coincide con la fecha fin de la ultima fila: 32/03/2025 16:43.,PROACTIVO,1.80
46,21796590,La fecha de inicio del parrafo en it_medidas_tomadas: ( 31/03/2025 15:49 ) no coincide con la fecha inicio de la penultima fila: None. La fecha de fin del parrafo en it_medidas_tomadas:( 31/03/2025 17:33 ) no coincide con la fecha fin de la ultima fila: None.,RECLAMO,1.80
47,21796622,La fecha/hora de fin del parrafo en MEDIDAS CORRECTIVAS:( 01/03/2025 10:15 ) no coincide con la columna FECHA Y HORA FIN DE EXCEL: 01/04/2025 10:15.,RECLAMO,1.80
